<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/Copy_of_autoencoder9_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=5a14d11b180a903bb5d2b00532237edc8510ba0c789cc140ea330f18ee7d2122
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [3]:
import re
import string
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os

# Set seed for PyTorch
torch.manual_seed(42)
# Set seed for NumPy
np.random.seed(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# upload HDFS dataset
!wget 'https://zenodo.org/record/3227177/files/HDFS_1.tar.gz'
!tar -xzvf "/content/HDFS_1.tar.gz" -C "/content/"   #unzip the file


--2024-01-03 13:48:40--  https://zenodo.org/record/3227177/files/HDFS_1.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/3227177/files/HDFS_1.tar.gz [following]
--2024-01-03 13:48:41--  https://zenodo.org/records/3227177/files/HDFS_1.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 161886385 (154M) [application/octet-stream]
Saving to: ‘HDFS_1.tar.gz’

HDFS_1.tar.gz       100%[===================>] 154.39M  17.3MB/s    in 10s     

2024-01-03 13:48:52 (15.0 MB/s) - ‘HDFS_1.tar.gz’ saved [161886385/161886385]

HDFS.log
anomaly_label.csv


In [5]:
# count rows of the raw log

with open('/content/HDFS.log', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

print('There are a total of {} lines'.format(totaln))

There are a total of 11175629 lines


In [6]:
def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message

    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

def find_blockid(text):
  match = re.search(r"blk_[-\w]+", text)
  return match.group()


In [7]:
def structured(address):
  with open(address, "r") as file:

      log2index = {}    # give for each unique cleaned log an index
      index2embed = {}  # give for each index(belongs to a unique cleaned log) sentence embedding of that unique cleaned log
      blockId_logs = {} # gather logs(index) which are blong to a specific block id
      index = -1

      for line in file:

        blockId = find_blockid(line)
        if not blockId in blockId_logs:
              blockId_logs[blockId] = []

        cleaned_line = clean(line)

        i = log2index.get(cleaned_line) # i is the index of the cleaned log
        if i is None:
            index += 1
            log2index[cleaned_line] = index
            index2embed[index] = model.encode(cleaned_line)
            blockId_logs[blockId].append(index)
        else :
            blockId_logs[blockId].append(i)


  embeddings = np.array(list(index2embed.values()))
  pca = PCA(n_components=10)
  pca.fit(embeddings)
  reduced_embeddings = pca.transform(embeddings)

  reduced_index2embed = {}
  for i,key in enumerate(index2embed.keys()):
    reduced_index2embed[key]=reduced_embeddings[i]

  return log2index, embeddings, reduced_embeddings, reduced_index2embed, blockId_logs


In [8]:
log2index, embeddings, reduced_embeddings, reduced_index2embed, blockId_logs = structured('/content/HDFS.log')


In [9]:
print('number of templates : ',len(log2index))

number of templates :  52


In [10]:
print('word embedding size(before pca) : ',embeddings[0].shape)
print('word embedding size(after pca) : ',reduced_embeddings[0].shape)

word embedding size(before pca) :  (384,)
word embedding size(after pca) :  (10,)


In [11]:
log2index

{'info dfs data node data xceiver receiving block src dest': 0,
 'info dfs fs namesystem block name systemallocate block': 1,
 'info dfs data node packet responder packet responder for block terminating': 2,
 'info dfs data node packet responder received block of size from': 3,
 'info dfs fs namesystem block name systemadd stored block block map updated is added to size': 4,
 'info dfs data node data xceiver received block src dest of size': 5,
 'info dfs data node data transfer transmitted block to': 6,
 'info dfs data node starting thread to transfer block to': 7,
 'info dfs fs namesystem block ask to replicate to datanode s': 8,
 'info dfs data node data xceiver served block to': 9,
 'info dfs data block scanner verification succeeded for': 10,
 'info dfs data node data xceiver write block received exception javanet socket timeout exception': 11,
 'info dfs data node packet responder packet responder exception javaio eof exception': 12,
 'info dfs data node data xceiver write block 

In [12]:
data_df = pd.DataFrame(list(blockId_logs.items()), columns=['BlockId', 'EventSequence'])
data_df.to_csv("HDFS_sequence.csv",index=None)

In [13]:
data_df.head()

,BlockId,EventSequence
0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ..."
1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 10,..."
2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ..."
3,blk_-9073992586687739851,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 10, 10..."
4,blk_7854771516489510256,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 10, 10..."


In [14]:
labels = pd.read_csv('/content/anomaly_label.csv')
labels.head()

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [15]:
# Merge the labels with the data_df DataFrame
data_df = data_df.merge(labels, on='BlockId', how='left')
data_df.head(3)

,BlockId,EventSequence,Label
0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ...",Normal
1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 10,...",Normal
2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ...",Anomaly


In [16]:
# Splitting the dataset into train and test
hdfs_sequence_normal = data_df[data_df['Label'] == 'Normal']

hdfs_sequence_abnormal = data_df[data_df['Label'] != 'Normal']

# Print the lengths of train and test datasets
print("Normal Dataset Length:", len(hdfs_sequence_normal))
print("Abnormal Test Dataset Length:", len(hdfs_sequence_abnormal))

Normal Dataset Length: 558223
Abnormal Test Dataset Length: 16838


In [17]:
hdfs_sequence_normal.head(3)

,BlockId,EventSequence,Label
0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ...",Normal
1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 10,...",Normal
3,blk_-9073992586687739851,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 10, 10...",Normal


In [18]:
hdfs_sequence_abnormal.head(3)

,BlockId,EventSequence,Label
2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ...",Anomaly
15,blk_-8531310335568756456,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 10, 10...",Anomaly
23,blk_3947106522258141922,"[0, 0, 1, 0, 2, 3, 2, 3, 4, 4, 2, 3, 4, 10, 10...",Anomaly


In [19]:
# Calculate the split indices based on the desired ratio
total_rows = len(hdfs_sequence_normal)
train_ratio = 0.95
train_rows = int(total_rows * train_ratio)

# Split the DataFrame into training and testing sets
train_hdfs_sequence_normal = hdfs_sequence_normal.iloc[:train_rows]
test_hdfs_sequence_normal = hdfs_sequence_normal.iloc[train_rows:]

print("Normal train Dataset Length:", len(train_hdfs_sequence_normal))
print("Normal Test Dataset Length:", len(test_hdfs_sequence_normal))


Normal train Dataset Length: 530311
Normal Test Dataset Length: 27912


In [55]:
import torch
import torch.nn as nn

class GRUAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, sequence_length, dropout_prob):
        super(GRUAutoencoder, self).__init__()
        self.sequence_length = sequence_length
        self.input_dim = input_dim

        # Encoder and Decoder GRU layers
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.projection_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        batch_size = x.shape[0]

        # Encoding phase
        _, hidden = self.encoder(x)

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, self.sequence_length, self.input_dim, device=x.device)

        # First input to the decoder is the zero tokens
        input_decoder = torch.zeros(batch_size, 1, self.input_dim, device=x.device)

        for t in range(1, self.sequence_length):
            output, hidden = self.decoder(input_decoder, hidden)
            output = self.projection_layer(output)
            outputs[:, t] = output.squeeze(dim=1)
            # Use actual input as the next input for the decoder (teacher forcing)
            input_decoder = x[:, t:t+1, :]
            #input_decoder = output

        return outputs


In [21]:
reduced_embeddings[0].shape[0]

10

In [56]:
# Define the dimensions
input_dim = reduced_embeddings[0].shape[0]
sequence_length = 5
hidden_dim = 32
dropout_prob = 0.1  # You can adjust this as needed

batch_size = 32

# Generating random input data for demonstration
input_data = torch.randn(batch_size, sequence_length, input_dim).to(device)

# Create the GRU Autoencoder model
model = GRUAutoencoder(input_dim, hidden_dim, sequence_length, dropout_prob).to(device)

# Forward pass through the model
output = model(input_data)

# Check the shape of the output
print("Output shape:", output.shape)


Output shape: torch.Size([32, 5, 10])


In [57]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 8778


In [58]:
name = 'logs_train'
window_size=sequence_length
num_sessions = 0
inputs = []
for log in train_hdfs_sequence_normal['EventSequence']:
   num_sessions += 1
   line = [ reduced_embeddings[int(i)] for i in log]
   for i in range(len(line) - window_size):
      inputs.append(line[i:i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))

dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 530311
Number of seqs(logs_train): 7705356


In [59]:
batch_size=256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
len(dataloader) # regarding to batch size

30100

In [60]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  input = seq[0].view(batch_size, -1)
  print(input.shape)
  break

torch.Size([256, 5, 10])
torch.Size([256, 50])


In [61]:
def adjust_learning_rate(optimizer, epoch, lr_step=(10,15,20,25), lr_decay_ratio=0.2):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 8
    elif epoch in [1, 2, 3]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

In [62]:
num_epochs = 20
learning_rate = 0.0005

# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
criterion = nn.MSELoss()

In [63]:
# Train the model
model.train()
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Epoch [1/20], train_loss: 0.0071
Epoch [2/20], train_loss: 0.0060
Epoch [3/20], train_loss: 0.0060
Epoch [4/20], train_loss: 0.0060
Epoch [5/20], train_loss: 0.0060
Epoch [6/20], train_loss: 0.0060
Epoch [7/20], train_loss: 0.0060
Epoch [8/20], train_loss: 0.0060


KeyboardInterrupt: ignored

In [45]:
torch.save(model.state_dict(), '/content/Autoencoder9_parameters.pth')

# **EVALUATION**


in cpu
```
my_model = model.load_state_dict(torch.load('model_parameters.pth', map_location=torch.device(device)))

# Put the model in evaluation mode if necessary
model.eval()
```

in cuda


```
model.load_state_dict(torch.load('model_parameters.pth'))

# Put the model in evaluation mode if necessary
model.eval()
```



In [64]:
reduced_embeddings.shape

(53, 10)

In [35]:
zero_log = np.array((np.zeros((1,reduced_embeddings[0].shape[0]))))
zero_log

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [36]:
reduced_embeddings = np.concatenate((reduced_embeddings, zero_log), axis=0)

In [37]:
reduced_embeddings[len(reduced_embeddings)-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [38]:
reduced_embeddings.shape

(53, 10)

In [65]:
def generate(data):
    window_size = sequence_length
    result = []
    for line in data['EventSequence']:
        line = line + [len(reduced_embeddings)-1] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by a template with zeros vector
        result.append(line)

    print('Number of sessions({}): {}'.format(name, len(result)))
    return result

In [66]:
test_normal_loader = generate(test_hdfs_sequence_normal)
test_abnormal_loader = generate(hdfs_sequence_abnormal)

Number of sessions(logs_train): 27912
Number of sessions(logs_train): 16838


In [67]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in test_normal_loader:
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [reduced_embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).to(device)
              output = model(seq.unsqueeze(0))

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += 1 # numbers of that set we have
                break
  with torch.no_grad():
      for line in test_abnormal_loader:
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [reduced_embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).to(device)
              output = model(seq.unsqueeze(0))

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += 1
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = len(test_abnormal_loader) - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [ ]:

threshold = [0.0093,0.0094,0.0095,0.0096]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.0093


In [71]:
threshold = [0.0092,0.0094,0.0096,0.0098]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.0092
elapsed_time: 827.752s
false positive (FP): 27912, false negative (FN): 140, Precision: 37.431%, Recall: 99.169%, F1-measure: 54.348%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0094
elapsed_time: 846.260s
false positive (FP): 59, false negative (FN): 145, Precision: 99.648%, Recall: 99.139%, F1-measure: 99.393%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.0096


KeyboardInterrupt: ignored